In [20]:
# imports
import pretty_midi
import warnings

### Example of MIDI file with empty tracks

In [34]:
black_or_white = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/Black or White copy.mid')

/Users/huongngo/opt/anaconda3/envs/gehirn/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [35]:
for instrument in black_or_white.instruments:
    print(instrument.notes)

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114), Note(start=19.396538, end=19.768304, pitch=80, velocity=111), Note(start=19.655158, end=19.951494, pitch=81, velocity=105), Note(start=19.913779, end=20.226278, pitch=80, velocity=99), Note(start=20.172399, end=20.452571, pitch=76, velocity=119), Note(start=20.431020, end=20.624985, pitch=71, velocity=115), Note(start=20.689640, end=20.975200, pitch=69, velocity=114), Note(start=20.948261, end=21.201493, pitch=68, velocity=91), Note(start=21.206881, end=21.384683, pitch=69, velocity=103), Note(start=21.465502, end=22.279078, pitch=71, velocity=118), Note(start=23.017225, end=23.103431, pitch=71, velocity=115), Note(start=23.141147, end=23.335112, pitch=71, velocity=116), Note(start=23.275845, end=23.556017, pitch=76, velocity=120), Note(start=23.529078, end=23.830801, pitch=80, velocity=113), Note(start=23.787698, end=24.137913, pitch=81, velocity=119), Note(star

In [37]:
black_or_white.key_signature_changes

[]

In [39]:
black_or_white.get_tempo_changes()

(array([0.]), array([116.00008507]))

A warning is returned stating that there is a tempo, key or time signature event on one of the tracks after the first one. Here, track is not referred to the MIDI file tracks, but the pretty_midi tracks. The first track always contains the metadata of the song (key signature, time signature etc.). It seems like there is an empty track when there is a tempo, key or time signature event on one of the tracks after the first one. 

The empty track can't be removed but we can flag and ignore that track. It can only be manually modified. 

### Example of same MIDI file that has been modified to not have empty tracks

In [36]:
black_or_white_m = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/black_or_white_m.mid')

In [40]:
for instrument in black_or_white_m.instruments:
    print(instrument.notes)

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114), Note(start=19.396538, end=19.768304, pitch=80, velocity=111), Note(start=19.655158, end=19.951494, pitch=81, velocity=105), Note(start=19.913778, end=20.226278, pitch=80, velocity=99), Note(start=20.172399, end=20.452571, pitch=76, velocity=119), Note(start=20.431020, end=20.624985, pitch=71, velocity=115), Note(start=20.689640, end=20.975200, pitch=69, velocity=114), Note(start=20.948260, end=21.201493, pitch=68, velocity=91), Note(start=21.206881, end=21.384683, pitch=69, velocity=103), Note(start=21.465502, end=22.279078, pitch=71, velocity=118), Note(start=23.017225, end=23.103431, pitch=71, velocity=115), Note(start=23.141147, end=23.335112, pitch=71, velocity=116), Note(start=23.275845, end=23.556017, pitch=76, velocity=120), Note(start=23.529078, end=23.830801, pitch=80, velocity=113), Note(start=23.787698, end=24.137913, pitch=81, velocity=119), Note(star

In [45]:
pretty_midi.key_number_to_key_name(black_or_white_m.key_signature_changes[0].key_number)

'C Major'

In [46]:
black_or_white_m.get_tempo_changes()

(array([0.]), array([116.00008507]))